In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
flight_2019 = pd.read_csv("/kaggle/input/flight-delay-prediction/Jan_2019_ontime.csv")
flight_2020 = pd.read_csv("/kaggle/input/flight-delay-prediction/Jan_2020_ontime.csv")

In [ ]:
## PROBLEM STATEMENT:

# Based on given data predict future flight delays for the month of January

In [ ]:
flight_2019.head()

In [ ]:
flight_2020.head()

In [ ]:
# Shape of datasets

In [ ]:
flight_2019.shape

In [ ]:
flight_2020.shape

In [ ]:
# Info on datasets:

In [ ]:
flight_2019.info()

In [ ]:
flight_2020.info()

In [ ]:
# Describe on datasets:

In [ ]:
flight_2019.describe().transpose()

In [ ]:
flight_2020.describe().transpose()

In [ ]:
# Dataset columns:

In [ ]:
flight_2019.columns

In [ ]:
flight_2020.columns

In [ ]:
# Both datasets have the same columns 

In [ ]:
# Checking for duplicate entries in dataset

In [ ]:
flight_2019.duplicated(keep = "first").value_counts()

In [ ]:
flight_2020.duplicated(keep = "first").value_counts()

In [ ]:
# Data Cleaning

In [ ]:
# Adding new "YEAR" parameter to the two datasets:
flight_2019["YEAR"] = 2019
flight_2020["YEAR"] = 2020

In [ ]:
# Adding new "MONTH" parameter to the two datasets:
# "MONTH" will be 1 as data is for the month of "JANUARY"
flight_2019["MONTH"] = 1
flight_2020["MONTH"] = 1

In [ ]:
# Thus, now we have "YEAR" , "MONTH" , "DAY_OF_MONTH", "DAY_OF_WEEK" : dates of the flight

In [ ]:
#  Renaming "DAY_OF_MONTH" as "DAY"
flight_2019.rename(columns = {"DAY_OF_MONTH" : "DAY"}, inplace= True)
flight_2020.rename(columns = {"DAY_OF_MONTH" : "DAY"}, inplace= True)



In [ ]:
flight_2019['DATE'] = pd.to_datetime(flight_2019[['YEAR','MONTH', 'DAY']])

In [ ]:
flight_2020['DATE'] = pd.to_datetime(flight_2020[['YEAR','MONTH', 'DAY']])

In [ ]:
flight_2019["DATE"].unique()

In [ ]:
flight_2020["DATE"].unique()

In [ ]:
flight_2019.columns

In [ ]:
# Handling "DEP_TIME" and "ARR_TIME" in both datasets:

# They are mentioned in "float64" format 

In [ ]:
import datetime

In [ ]:
def format_time(t):
    if pd.isnull(t):
        return np.nan
    else:
        if t == 2400: t = 0
        t = "{0:04d}".format(int(t))
        result = datetime.time(int(t[0:2]), int(t[2:4]))
        return result

In [ ]:
# Change format of arrival time and departure time in both datasets :

In [ ]:
flight_2019['DEP_TIME'] = flight_2019['DEP_TIME'].apply(format_time)
flight_2020['DEP_TIME'] = flight_2020['DEP_TIME'].apply(format_time)
flight_2019['ARR_TIME'] = flight_2019['ARR_TIME'].apply(format_time)
flight_2020['ARR_TIME'] = flight_2020['ARR_TIME'].apply(format_time)


In [ ]:
# columns in changed format

In [ ]:
flight_2019["DEP_TIME"]

In [ ]:
flight_2019["ARR_TIME"]

In [ ]:
flight_2020["DEP_TIME"]

In [ ]:
flight_2020["ARR_TIME"]

In [ ]:
flight_2019.head()

In [ ]:
flight_2020.head()

In [ ]:
## Concatenate the two datasets:

flight = pd.concat([flight_2019, flight_2020])

In [ ]:
flight.shape

In [ ]:
# info on flight data
flight.info()

In [ ]:
# describe on flight data
flight.describe()

In [ ]:
# check null values
flight.isna().sum()

In [ ]:
# null values in percentage 

((flight.isna().sum()/len(flight))*100).round(2)

In [ ]:
## Dropping "Unnamed: 21" as all its entries are empty
flight = flight.drop(columns = ["Unnamed: 21"])

In [ ]:
# Dropping rows for which columns have very less percentage of null values ( < 3%)
flight = flight.dropna()

In [ ]:
## Now our data has no null entries

In [ ]:
flight.nunique()

In [ ]:
# find and remove nunique == 1
flight.nunique()[flight.nunique() == 1].index.to_list()

In [ ]:
# Two constant feature columns found : "CANCELLED" and "DIVERTED"
flight = flight.drop(columns = ["CANCELLED", "DIVERTED"])

In [ ]:
flight.columns.size

In [ ]:
# value_counts() for columns[0:10]
i=0
for col in flight.columns[:10]:
  print("Columns no is: " + str(i))
  i = i+1
  print("Column is : " + col)
  print(flight[col].value_counts())
  print()
  print()

In [ ]:
# value_counts() for columns[10:19]
for col in flight.columns[10:]:
  print("Column no is: " + str(i))
  i = i+1
  print("Column is : " + col)
  print(flight[col].value_counts())
  print()
  print()

In [ ]:
## Here, from above we can conclude the following:

# "OP_CARRIER" and "OP_UNIQUE_CARRIER" refect the same category of information
# -- one of them can be dropped
# "OP_CARRIER" and "OP_CARRIER_AIRLINE_ID" also represent the same information
# -- all categories and their counts are same for both columns

## "ORIGIN_AIRPORT_ID" and "ORIGIN" also represent same information 
# It is inherently categorical in nature -- hence, we drop "ORIGIN_AIRPORT_ID"

## "DEST_AIRPORT_ID" and "DEST" also represent same information
# It is also inherently categorical in nature -- hence, we drop "DEST_AIRPORT_ID"

In [ ]:
flight = flight.drop(columns = ["OP_UNIQUE_CARRIER", "OP_CARRIER_AIRLINE_ID", "ORIGIN_AIRPORT_ID", "DEST_AIRPORT_ID"])

flight.columns

In [ ]:
## "MONTH" , "YEAR", "DAY" are not required

flight = flight.drop(columns = ["MONTH", "YEAR", "DAY"])

In [ ]:
# Renaming :

flight.rename(columns = {"DEP_DEL15" : "DEP_DELAY"}, inplace= True)
flight.rename(columns = {"ARR_DEL15" : "ARR_DELAY"}, inplace= True)


In [ ]:
## Removing "ARR_TIME" and "ARR_DELAY" as "DEP_DELAY" is considered as TARGET variable

In [ ]:
flight = flight.drop(columns= ["ARR_DELAY", "ARR_TIME"])

In [ ]:
## DATA VISUALIZATION:

In [ ]:
## Analysis of target variable

flight["DEP_DELAY"].value_counts()

In [ ]:
# Plot to visualize the target variable in the dataset:

sns.set(style="darkgrid")

plt.figure(figsize=(10,6))
ax = sns.countplot(x="DEP_DELAY", data=flight)
plt.title('Distribution of target variable')
plt.xlabel('0 - NO DEPARTURE DELAY  |  1 -DEPARTURE DELAY')
plt.ylabel('Frequency')

for p in ax.patches:
        ax.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+0.1, p.get_height()+50))

In [ ]:
# Percentage of True/False for Target Variable

print("Percentage of No Departure Delay")
print(((flight["DEP_DELAY"].value_counts()[0]/flight.shape[0])*100).round(2))
print()
print("Percentage of Departure Delay")
print(((flight["DEP_DELAY"].value_counts()[1]/flight.shape[0])*100).round(2))

In [ ]:
# Pie chart for the TARGET variable "DEP_DELAY"

plt.figure(figsize = (7, 7))
flight["DEP_DELAY"].value_counts().plot.pie()
plt.show()

# 0 -> NO DELAY 
# 1 -> DELAY

In [ ]:
flight.corr()

In [ ]:
# Plotting the correaltion data heatmap

plt.figure(figsize = (4,4))
sns.heatmap(flight.corr())
plt.show()

In [ ]:
# Model Building :

In [ ]:
# Dropping columns 
flight = flight.drop(columns = ["TAIL_NUM", "ORIGIN_AIRPORT_SEQ_ID", "DEST_AIRPORT_SEQ_ID"])

In [ ]:
flight.head()

In [ ]:
## We will require the day_of_month column only out of the date column , as year is in two categories only
# and month is january "1" for all rows hence, that is not a factor

flight["DAY_OF_MONTH"] = flight["DATE"].dt.day

In [ ]:
## Drop "DATE" feature:
flight = flight.drop(columns = "DATE")

In [ ]:
flight.head()

In [ ]:
## Moving "DEP_DELAY" column and making it the last colunm in the dataset:
cols_at_end = ["DEP_DELAY"]
flight_data = flight[[c for c in flight if c not in cols_at_end] 
        + [c for c in cols_at_end if c in flight]]

In [ ]:
flight_data.head()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
## Selecting the cyclic features in the datasets: They need to be encoded separately

In [ ]:
def encode(data, col, max_val):
    data.loc[:, col + '_SIN'] = np.sin(2 * np.pi * data[col]/max_val)
    data.loc[:, col + '_COS'] = np.cos(2 * np.pi * data[col]/max_val)
    return data


In [ ]:
encode( flight_data, "DAY_OF_WEEK", 7)
encode( flight_data, "DAY_OF_MONTH", 30)

In [ ]:
flight_data.columns

In [ ]:
## Now, we can drop the original "DAY_OF_WEEK" and "DAY_OF_MONTH" columns from the dataset
flight_data = flight_data.drop(columns = ["DAY_OF_WEEK", "DAY_OF_MONTH"])

In [ ]:
flight_data.dtypes

In [ ]:
flight_data["DEP_TIME"]

In [ ]:
## We have departure time in 24 hour format 
## -- In order to apply cyclic encoding , we will have to convert it into minutes past midnight
## -- and then apply cyclic encoding to the dataset

In [ ]:
# Extract hour from the "DEP_TIME" using str
flight_data["DEP_TIME"].astype(str).str[0:2]

In [ ]:
# Extract minutes from the "DEP_TIME" using str
flight_data["DEP_TIME"].astype(str).str[3:5]

In [ ]:
# "DEP_TIME" converted into minutes past midnight

((flight_data["DEP_TIME"].astype(str).str[0:2]).astype(int))*60 + (flight_data["DEP_TIME"].astype(str).str[3:5].astype(int))

In [ ]:
dep_time_in_minutes = ((flight_data["DEP_TIME"].astype(str).str[0:2]).astype(int))*60 + (flight_data["DEP_TIME"].astype(str).str[3:5].astype(int))

In [ ]:
def add_new_col(data, col, new_col):
    data.loc[:, col + '_MINUTES'] = new_col
    return data

In [ ]:
## Creating "DEP_TIME_MINUTES" in terms of minutes_data:
add_new_col(flight_data, "DEP_TIME", dep_time_in_minutes)

In [ ]:
max_time_in_a_day = 24*60

encode( flight_data, "DEP_TIME_MINUTES", max_time_in_a_day)


In [ ]:
flight_data.columns

In [ ]:
## Columns added with sin and cos parameters w.r.t "DEP_TIME_MINUTES" column

In [ ]:
## Now, we can drop "DEP_TIME" column and "DEP_TIME_MINUTES"
flight_data = flight_data.drop(columns = ["DEP_TIME", "DEP_TIME_MINUTES"])

In [ ]:
## Dtypes of columns present in the dataset at this point
flight_data.dtypes

In [ ]:
## Applying LabelEncoding on "ORIGIN" and "DEST" columns:

flight_data["ORIGIN"] = flight_data["ORIGIN"].astype('category')
flight_data["DEST"] = flight_data["DEST"].astype('category')

flight_data["ORIGIN_ENCODE"] = flight_data["ORIGIN"].cat.codes
flight_data["DEST_ENCODE"] = flight_data["DEST"].cat.codes

## Drop "ORIGIN" and "DEST" columns 
flight_data = flight_data.drop(columns = ["ORIGIN", "DEST"])

In [ ]:
# One Hot encoding
flight_data = pd.get_dummies(data = flight_data, columns = ["OP_CARRIER", "DEP_TIME_BLK"], drop_first = True)
flight_data.head()

In [ ]:
# Moving "DEP_DELAY" column and making it the last colunm in the dataset:
cols_at_end = ["DEP_DELAY"]
flight_data = flight_data[[c for c in flight_data if c not in cols_at_end] 
        + [c for c in cols_at_end if c in flight_data]]

In [ ]:
flight_data.dtypes

In [ ]:
# Loading X and y parameters for our model building

X = flight_data.iloc[:, :-1].values
y = flight_data.iloc[:, -1].values

In [ ]:
# Split data 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
# find index of columns to be scaled 
print(flight_data.columns.get_loc("OP_CARRIER_FL_NUM"))
print(flight_data.columns.get_loc("DISTANCE"))
print(flight_data.columns.get_loc("ORIGIN_ENCODE"))
print(flight_data.columns.get_loc("DEST_ENCODE"))


In [ ]:
# Applying Feature Scaling 

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, [0, 1, 8, 9]] = sc.fit_transform(X_train[:, [0, 1, 8, 9]])
X_test[:, [0, 1, 8, 9]] = sc.transform(X_test[:, [0, 1, 8, 9]])


In [ ]:
# Model 1: Logistic Regression

from sklearn.linear_model import LogisticRegression

log_classifier = LogisticRegression(random_state = 0)
log_classifier.fit(X_train, y_train)

In [ ]:
# Predicting the test set result : Logistic Classifier

y_pred = log_classifier.predict(X_test)

In [ ]:
# Making the confusion matrix 

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix : Logistic Regression")
print(cm)
print()
print("Accuracy Score : Logistic Regression")
print( accuracy_score(y_test, y_pred))


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
# Model 2: Naive Bayes Algorithm

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

In [ ]:
# Predicting the test set result : Naive Bayes

y_pred_nb = nb_classifier.predict(X_test)

In [ ]:
# Making the confusion matrix : Naive Bayes

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred_nb)
print("Confusion Matrix : Naive Bayes ")
print(cm)
print()
print("Accuracy Score : Naive Bayes ")
print( accuracy_score(y_test, y_pred_nb))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_nb))

In [ ]:
# Model 3: DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
dt_classifier.fit(X_train, y_train)

In [ ]:
# Predicting the test set result : Decision Tree Classifier

y_pred_dt = dt_classifier.predict(X_test)

In [ ]:
# Making the confusion matrix : Decision Tree Classifier

cm = confusion_matrix(y_test, y_pred_dt)
print("Confusion Matrix : Decision Tree Classifier ")
print(cm)
print()
print("Accuracy Score : Decision Tree Classifier ")
print( accuracy_score(y_test, y_pred_dt))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_dt))

In [ ]:
## Applying K-fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies_decision_tree = cross_val_score(estimator = dt_classifier, X = X_train, y = y_train, cv = 10)
print("Accuracies mean for Decision tree is : ")
print(accuracies_decision_tree.mean())
print()
print("Standard Deviation for Decision Tree accuracies is :")
print(accuracies_decision_tree.std())

In [ ]:
# Model 4: Random Forest Classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
rf_classifier.fit(X_train, y_train)

In [ ]:
# Predicting the test set result : Random Forest Classifier

y_pred_rf = rf_classifier.predict(X_test)

In [ ]:
# Making the confusion matrix : Random Forest Classifier

cm = confusion_matrix(y_test, y_pred_rf)
print("Confusion Matrix : Random Forest Classifier ")
print(cm)
print()
print("Accuracy Score : Random Forest Classifier ")
print( accuracy_score(y_test, y_pred_rf))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_rf))

In [ ]:
## Applying K-fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies_random_forest = cross_val_score(estimator = rf_classifier, X = X_train, y = y_train, cv = 10)
print("Accuracies mean for Random Forest is : ")
print(accuracies_random_forest.mean())
print()
print("Standard Deviation for Random Forest accuracies is :")
print(accuracies_random_forest.std())